In [10]:
from linkedin_scraper import Person,actions,Company
from selenium.webdriver import Chrome
import os,dotenv
from selenium.webdriver.edge.options import Options
dotenv.load_dotenv()



#options = Options()
#options.add_argument("headless")  # Enable headless mode
#options.add_argument("window-size=1920,1080")  # Optional: set window size if needed

driver = Chrome()
actions.login(driver=driver,email=os.getenv("LINKEDIN_USER"),password=os.getenv("LINKEDIN_PASSWORD"))


In [2]:

#rick_fox = Person("https://www.linkedin.com/in/rifox?trk=pub-pbmap",driver=driver)
iggy = Person("https://www.linkedin.com/in/andre-iguodala-65b48ab5",driver=driver,scrape=False,get=False)
#Anirudra = Person("https://in.linkedin.com/in/anirudra-choudhury-109635b1",driver=driver)

In [3]:
import os
from typing import List
from time import sleep
import urllib.parse

from linkedin_scraper.objects import Scraper
from linkedin_scraper import constants as c
from linkedin_scraper.jobs import Job

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [4]:
class Event(Scraper):
    def __init__(
        self,
        linkedin_url=None,
        event_title=None,
        organizer=None,
        organizer_linkedin_url=None,
        location=None,
        event_date=None,
        attendees_count=None,
        description=None,
        driver=None,
        close_on_complete=True,
        speakers = [],
        event_link = None,
        scrape=True,
    ):
        super().__init__()
        self.linkedin_url = linkedin_url
        self.event_title = event_title
        self.driver = driver
        self.organizer:Company = organizer
        self.location = location
        self.attendees_count = attendees_count
        self.description = description
        self.event_date = event_date
        self.speakers:list[Person] = speakers
        self.event_link = event_link

        if scrape:
            self.scrape(close_on_complete)


# Get Event list

In [23]:
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import NoSuchElementException , ElementNotInteractableException,TimeoutException
import time
import datetime
iggy.driver = driver
event: WebElement
MIN_ATENDEES = 50
events = []
for page in range(10):
    driver.get(f"https://www.linkedin.com/search/results/events/?keywords=Networking june&page={page}")
    try:
        iggy.wait_for_element_to_load(by=By.TAG_NAME, name="main")
        time.sleep(1)
        for atendees in driver.find_elements(By.CLASS_NAME,"entity-result__insights"):

            event = atendees.find_element(By.XPATH,"..")
            atendees = int(atendees.text.split(" ")[0].strip())
            url = event.find_element(By.TAG_NAME,"a").get_attribute("href")
            if atendees > MIN_ATENDEES:
                events.append(Event(
                    linkedin_url=url,
                    scrape=False,
                    driver=driver,
                    attendees_count=atendees
                ))
    except NoSuchElementException :
         break

    
    

# Event scraping

In [27]:
import tqdm
event:Event
for event in tqdm.tqdm(events):
    if event.organizer is not None:
        continue
    try:
        driver.get(event.linkedin_url)
    except:
        driver.get(event.linkedin_url)
    iggy.wait_for_element_to_load(by=By.TAG_NAME, name="main")
    time.sleep(1)
    try:
        see_more = driver.find_element(By.CSS_SELECTOR, 'a[role="button"].lt-line-clamp__more')
        parent = see_more.find_element(By.XPATH,"..").find_element(By.XPATH,"..").find_element(By.XPATH,"..")
        see_more.click()

        event.description = parent.text.removesuffix("See less").strip()
    except ElementNotInteractableException:
        pass
    except NoSuchElementException:
        pass
    header = driver.find_element(By.TAG_NAME,"h1")
    event.event_title = header.text.strip()
    header = header.find_element(By.XPATH,"..")
    # Magic numbers go brrrr
    organizer_url = header.find_element(By.TAG_NAME,"a").get_attribute("href")
    if "company" in organizer_url:
        event.organizer = Company(organizer_url,scrape=False,get=False,driver=driver)
    else:
        event.organizer = Person(organizer_url,scrape=False,get=False,driver=driver)
    event.event_date = header.find_element(By.CLASS_NAME,"mr2").text
    try:
        event.location = header.find_element(By.TAG_NAME,"p").text
    except NoSuchElementException:
        event.location=None
    event.event_link= header.find_elements(By.TAG_NAME,"a")[-1].get_attribute("href")

    try:
        event.speakers = []
        for speaker in driver.find_element(By.CLASS_NAME,"events-live-speaker-list__speaker-container").find_elements(By.TAG_NAME,"section"):
            event.speakers.append(Person(speaker.find_element(By.TAG_NAME,"a").get_attribute("href"),
                                driver=driver,
                                get=False,
                                scrape=False))
    except NoSuchElementException:
        pass

100%|██████████| 14/14 [00:00<?, ?it/s]


In [31]:
event:Event

driver = Chrome()
actions.login(driver=driver,email=os.getenv("LINKEDIN_USER"),password=os.getenv("LINKEDIN_PASSWORD"))
for event in tqdm.tqdm(events):
    try:
        event.driver = driver
        if event.organizer:
            event.organizer.driver = driver
            driver.get(event.organizer.linkedin_url)
            if isinstance(event.organizer,Company):
                event.organizer.scrape(get_employees=False,close_on_complete=False)
            elif isinstance(event.organizer,Person):
                event.organizer.scrape(close_on_complete=False)
        if len(event.speakers) > 0:
            for speaker in event.speakers:
                driver.get(speaker.linkedin_url)
                speaker.scrape()
    except TimeoutError:
        continue

  0%|          | 0/14 [00:00<?, ?it/s]

<selenium.webdriver.remote.webelement.WebElement (session="51ab0b5933e28de5f3a4e55e8ef61ad2", element="f.382CB2847C205302756CF017EC3BEE14.d.9238F4EE1FCA18CE9E3427DB84595E06.e.254")>


  0%|          | 0/14 [00:17<?, ?it/s]


KeyboardInterrupt: 

In [35]:
event:Event
results = []
speakers = []
for i, event in enumerate(events):
    result = {
        "id":i,
        "name":event.event_title,
        "date":event.event_date,
        "event_url":event.event_link,
        "atendees":event.attendees_count,
        "organizer":event.organizer.name,
        "organizer_url":event.organizer.linkedin_url,

    }
    results.append(result)
    speaker: Person
    for speaker in event.speakers:
        speakers.append({
            "event_id":i,
            "speaker_url":speaker.linkedin_url
            })

   

In [38]:
import pandas as pd

pd.DataFrame(results).to_csv("linkedin_scrape_result.csv")
pd.DataFrame(speakers).to_csv("linkedin_speakers_scrape_result.csv")